<a href="https://colab.research.google.com/github/mrklees/PracticalStatistics/blob/master/Regression_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression Modelling

### Don't Forget to Run All (Ctrl+F9)
### Alex! Press the Record Button!!!


By the end of the session participants will...

* Review Expectation and Variance
* Review the concept of a joint distribution
* 

## Language Repository
These are some key terms that I will throw around a lot.  You can always review their definitions here or [go back to the notebook on Statistical Testing & p-values to review.](https://colab.research.google.com/drive/15GQcjwz1TlVOOZfxiYakS1fAl3-3BiJA)

**Expected Value: ** This is the average! It's the long-run average value of repetitions of the same experiment. 

**Variance:** This is a measure of how spread out a distribution is from its mean.  The greater the variance, the futher values fall from its mean.  

**Standard Deviation:** This is really just the square root of the variance: $\text{std dev}(X) = \sqrt{Var(X)}$, and thus it behaves very similarly to variance.  The greater the standard deviation, the futher values fall from its mean.  It is often used in the context of normally distributed data because of how nicely a standard deviation partitions the normal distribution, per the obligatory graph:

![Obligatory Graph](https://www.biologyforlife.com/uploads/2/2/3/9/22392738/sd2_orig.png)



## Motivating Today with Simpson's Paradox

In the last session, we learned about methods which allows us to directly compare two different variables, like focus list status and assessment scores.  And, ignore the fact that I spoke at length about why you shouldn't really use those mothods, you might wonder why we don't just stop there.  Why work up to more complex models?

We will start off with a small example that gets at this idea.  It involves a very simple table of data.  Suppose we did some trial on the effectiveness of a drug on the risk of heart attacks. The data is summarized below.

| | Control - Heart Attacks | Control - Total Participants | Treatment - Heart Attacks | Treatment - Total Participants+ |
|:---:|:---:|:---:|:---:|:---:|
|Female|1|20|3|40|
|Male|12|40|8|20|

The paradox here comes from the two stories that I can tell with this table of data.  

The **first story** is about a fantastic new drug! In a clinical trial of 120 participants, participants who tool the experimental drug experienced heart attacks had about a 15% reduction in heart attacks.  Talk to your doctor today!

However the **second story** is an investigative journlism piece about a dangerous drug put out on the market.  When *looking at men and women separately* it turns that in either case it *raises your risk of heart attack* by about 25% (for men) to 50% (for women). Obviously a drug which is bad for both men and women should be banned. 

To be clear, these calculations are really just simple percentages. In the first story, we just pretend like the data isn't segmented.  So $\frac{13}{60}=21.6\%$ participants in the control and $\frac{11}{60}=18.3\%$ in the treatment group experienced heart attacks. Of course I then presented their ratio, because it's a much more marketable number 👍.

Since the second story is symmetrical, I'll just talk about women.  In the control group, $\frac{1}{20}=5\%$ of women experience heart attacks where as $\frac{3}{40}=7.5\%$ women in the treatment group experienced heart attacks. A similar increase was seen for men. 

There is a lot within Simpson's Paradox.  It was published over 60 years ago and still is the topic of some conversation. What I really want you to get from this is: this is what confounding can look like.  If we just measure effects directly then there is a risk that what we measure could not only be different from the true effect, but could be in the completely wrong direction. 

In [0]:
#@title Imports and Global Variables (run this cell first)  { display-mode: "form" }
#@markdown This sets the warning status (default is `ignore`, since this notebook runs correctly)
warning_status = "ignore" #@param ["ignore", "always", "module", "once", "default", "error"]
import warnings
warnings.filterwarnings(warning_status)
with warnings.catch_warnings():
    warnings.filterwarnings(warning_status, category=DeprecationWarning)
    warnings.filterwarnings(warning_status, category=UserWarning)

import numpy as np
import pandas as pd
import os
#@markdown This sets the styles of the plotting (default is styled like plots from [FiveThirtyeight.com](https://fivethirtyeight.com/))
matplotlib_style = 'fivethirtyeight' #@param ['fivethirtyeight', 'bmh', 'ggplot', 'seaborn', 'default', 'Solarize_Light2', 'classic', 'dark_background', 'seaborn-colorblind', 'seaborn-notebook']
import matplotlib.pyplot as plt; plt.style.use(matplotlib_style)
%matplotlib inline
import seaborn as sns; sns.set_context('notebook')

import statsmodels.api as sm
from statsmodels.api import OLS

In [49]:
#@title Read the Data from the Web { display-mode: "form" }
data_url = 'https://impactblob.blob.core.windows.net/public/anon_hmh.csv'
data = pd.read_csv(data_url)
data.head()

,GRADE_ID_NUMERIC,OFFICIALFLLIT,OFFICIALFLMTH,FL_LIT_MET_DOSAGE,FL_MTH_MET_DOSAGE,litassess_pre_value_num,LITASSESS_RAWCHANGE,LITASSESS_SRITARGET,mathassess_pre_value_num,MathAssess_RAWCHANGE,SMI_TARGET,AnonId,SiteId,SchoolId,att_pre_value,att_post_value
0,7,0,0,NaN,NaN,NaN,NaN,NaN,505.0,130.0,150.0,2634048971,1,59,NaN,NaN
1,7,0,0,NaN,NaN,NaN,NaN,NaN,505.0,290.0,150.0,2405496161,1,59,NaN,NaN
2,7,0,0,NaN,NaN,NaN,NaN,NaN,610.0,305.0,150.0,2627617999,1,59,NaN,NaN
3,8,0,0,NaN,NaN,NaN,NaN,NaN,695.0,135.0,150.0,1437215014,1,59,NaN,NaN
4,8,0,0,NaN,NaN,NaN,NaN,NaN,610.0,75.0,150.0,461103962,1,59,NaN,NaN


In [50]:
#@title Descriptive Stats to Reference { display-mode: "form" }
def describe_nulls(data):
    desc = data.describe(include=data.dtypes.unique())
    desc.loc['% Null'] = data.isna().sum() / data.shape[0]
    return desc
describe_nulls(data)

,GRADE_ID_NUMERIC,OFFICIALFLLIT,OFFICIALFLMTH,FL_LIT_MET_DOSAGE,FL_MTH_MET_DOSAGE,litassess_pre_value_num,LITASSESS_RAWCHANGE,LITASSESS_SRITARGET,mathassess_pre_value_num,MathAssess_RAWCHANGE,SMI_TARGET,AnonId,SiteId,SchoolId,att_pre_value,att_post_value
count,6620.000000,6620.000000,6620.000000,2651.000000,2784.000000,4478.000000,3762.000000,4123.000000,4633.000000,3879.000000,3262.000000,6.620000e+03,6620.000000,6620.000000,1290.000000,1156.000000
mean,7.344411,0.401964,0.421148,0.836288,0.858477,596.491291,53.556087,105.724715,459.830563,81.768497,126.587983,2.152367e+09,1.659517,37.133082,0.903720,0.888573
std,1.944957,0.490332,0.493781,0.370084,0.348623,341.226084,159.348106,82.842579,245.442909,207.451570,57.186226,1.239760e+09,1.086982,18.407459,0.087059,0.105393
min,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-747.000000,3.000000,-300.000000,-1070.000000,20.000000,7.713890e+05,1.000000,1.000000,0.300000,0.211000
25%,6.000000,0.000000,0.000000,1.000000,1.000000,350.250000,-25.000000,51.000000,270.000000,-45.000000,90.000000,1.062132e+09,1.000000,21.000000,0.868500,0.844000
50%,8.000000,0.000000,0.000000,1.000000,1.000000,648.000000,41.000000,70.000000,455.000000,95.000000,100.000000,2.159307e+09,1.000000,43.000000,0.923500,0.921000
75%,9.000000,1.000000,1.000000,1.000000,1.000000,848.750000,138.750000,133.000000,640.000000,215.000000,150.000000,3.228741e+09,2.000000,51.000000,0.966000,0.960000
max,10.000000,1.000000,1.000000,1.000000,1.000000,1540.000000,1048.000000,364.000000,1190.000000,845.000000,260.000000,4.294937e+09,5.000000,65.000000,1.000000,1.000000
% Null,0.000000,0.000000,0.000000,0.599547,0.579456,0.323565,0.431722,0.377190,0.300151,0.414048,0.507251,0.000000e+00,0.000000,0.000000,0.805136,0.825378


# Regression

This is a typical section where people bring out formulas and abstract graphs to try to explain this topic... but in the flavor of practical statistics we're going to talk about regression in terms of a different conceptual model.  One which will hopefully allow us to worry a little bit less about the math.  The reason that I feel that I can get away with this is that *most modern statistical interfaces* allow you to operate at this level of abstraction with this kind of conceptual framework instead of having to concern yourself with the details. Of course there is some variation in the aesthetics, but largely they are all the same.

## Regression as a Framework for Prediction

In our framework, regression is all about being able to take some data and make predictions about future data. In this context, we'll use a few new pieces of language with somewhat specific meanings.  Let's go through the language with some explanation. 

**Features**: The data we use to make predictions.  For a student, this might be things like grade, school, focus list student, etc...  We hope that this data contains information about the **outcome** we want to predict.

**Outcomes**: The variable we want to make predictions about.  For example, we've made predictions about how much a student will improve on a math assessment. 

**Model**:  A **model** is used to make predictions about **outcomes**. Generally speaking, it is simply some mathematical device which tells us how to multiply and add our data to get **predictions** about our **outcomes**. 

The general goal of Regression is to *fit* (or *train*) **models** on **features** where the **outcomes** are known, and then use them to make predictions on new data where the **outcomes** aren't known.  In the last session we previewed a formula notation which expresses this idea.  In its full form it might look like:
$$ \text{Outcomes} \sim \text{Model(Features)}$$
Although, we dropped the model portion, because it doesn't really tell us much.  So we're just left with:
$$ \text{Outcomes} \sim \text{Features}$$

### Fitting Model: What do we need to understand for Practical Statistics?

For the purposes of Pratical Statistics, we will acknowledge that these models exist and talk *exclusively* about how to use them.  Unfortunately, going much further into how a model is trained requires going into some calculus, so we will skate around the topic.  Instead, since you can use software prepared by professionals you can largely trust that *as long as you specify the model correctly*, the software will not make a mistake in fitting the model correctly. What I mean by *specifying the model correctly* will be one of the key subjects we discuss. 

In [67]:
#@title Regression Choose Your Own Adventure {run: 'auto', display-mode: "form"} 
#@markdown This tool will allow you fit nearly any possible linear model from the data.  Start by selecting which column will be the **Outcome**.  I would recommend `MathAssess_RAWCHANGE` or `LITASSESS_RAWCHANGE`, but I've left every column available. 
Outcomes = "MathAssess_RAWCHANGE" #@param ['GRADE_ID_NUMERIC', 'OFFICIALFLLIT', 'OFFICIALFLMTH', 'FL_LIT_MET_DOSAGE', 'FL_MTH_MET_DOSAGE', 'litassess_pre_value_num', 'LITASSESS_RAWCHANGE', 'LITASSESS_SRITARGET', 'mathassess_pre_value_num', 'MathAssess_RAWCHANGE', 'SMI_TARGET', 'AnonId', 'SiteId', 'SchoolId', 'att_pre_value', 'att_post_value']

#@markdown Then select which columns of data should be included in your model!  Sorry for the rough interface, but Colab hasn't published a better one yet. 
GRADE_ID_NUMERIC = False #@param {type:"boolean"}
OFFICIALFLLIT = False #@param {type:"boolean"}
OFFICIALFLMTH = False #@param {type:"boolean"}
FL_LIT_MET_DOSAGE = False #@param {type:"boolean"}
FL_MTH_MET_DOSAGE = False #@param {type:"boolean"}
litassess_pre_value_num = False #@param {type:"boolean"}
LITASSESS_RAWCHANGE = False #@param {type:"boolean"}
LITASSESS_SRITARGET = False #@param {type:"boolean"}
mathassess_pre_value_num = False #@param {type:"boolean"}
MathAssess_RAWCHANGE = False #@param {type:"boolean"}
SMI_TARGET = False #@param {type:"boolean"}
SiteId = False #@param {type:"boolean"}
SchoolId = False #@param {type:"boolean"}
att_pre_value = True #@param {type:"boolean"}
att_post_value = False #@param {type:"boolean"}

colnames= np.array([
           'GRADE_ID_NUMERIC', 'OFFICIALFLLIT', 'OFFICIALFLMTH',
           'FL_LIT_MET_DOSAGE', 'FL_MTH_MET_DOSAGE', 'litassess_pre_value_num',
           'LITASSESS_RAWCHANGE', 'LITASSESS_SRITARGET',
           'mathassess_pre_value_num', 'MathAssess_RAWCHANGE', 'SMI_TARGET',
           'SiteId', 'SchoolId', 'att_pre_value', 'att_post_value'])

responses = np.array([GRADE_ID_NUMERIC, OFFICIALFLLIT, OFFICIALFLMTH,
FL_LIT_MET_DOSAGE, FL_MTH_MET_DOSAGE, litassess_pre_value_num,
LITASSESS_RAWCHANGE, LITASSESS_SRITARGET,
mathassess_pre_value_num, MathAssess_RAWCHANGE, SMI_TARGET,
SiteId, SchoolId, att_pre_value, att_post_value])

# Get Selected Features
Features = list(colnames[responses])

try:
    assert Outcomes not in Features
    
    model_string = Outcomes + " ~ " + ' + '.join(Features)
    print(f"Current Model: {model_string}")
    print("Fitting... Be aware that null data will be dropped...")

    model = OLS(endog=data[Outcomes], exog=sm.add_constant(data[Features]), missing='drop').fit()
    print("Done... Returning Summary...")
    print(model.summary())
except AssertionError:
    print("Your outcome variable is also selected as a feature you silly goose!")

Current Model: MathAssess_RAWCHANGE ~ att_pre_value
Fitting... Be aware that null data will be dropped...
Done... Returning Summary...
                             OLS Regression Results                             
Dep. Variable:     MathAssess_RAWCHANGE   R-squared:                       0.004
Model:                              OLS   Adj. R-squared:                  0.003
Method:                   Least Squares   F-statistic:                     3.991
Date:                  Mon, 15 Apr 2019   Prob (F-statistic):             0.0460
Time:                          21:38:29   Log-Likelihood:                -7164.6
No. Observations:                  1053   AIC:                         1.433e+04
Df Residuals:                      1051   BIC:                         1.434e+04
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                    coef    std err          t      P>|

![Makes all the models](https://i.imgflip.com/2ynvwj.jpg)
![Which one is right though?](https://i.imgflip.com/2ynw4z.jpg)

# All Models Are Wrong

Now that we have the ability to generate in fact thousands of different models, we need some method of choosing between them. It's at about this point that I should offer a piece of wisdom [attributed to statistician George Box](https://en.wikipedia.org/wiki/All_models_are_wrong): *"All models are wrong, but some are useful."*  We aren't fortune tellers.  As much as we would like, we cannot predict the future and our puny mathematics cannot fully represent natural processes (yet).  So we start from the premise that our models are at best approximations, but thankfully the full effort of statisticians in the last century have gone into showing that we can build some good approximations and that there are some processes we can utilize to get there.

## Process for Finding Good Models

Finding good models is ultimately a iterative process.  Like a good scientist, we have to be consistently skeptical of our current model, knowing that there is probably a better one out there. I like thinking about this as a process, summed up in this short diagram:

![Box's Loop](https://cdn-images-1.medium.com/max/800/0*k1g-sYQ0QTOtOAyK.png)

The [developers of Edward call this Box's Loop](http://www.cs.columbia.edu/~blei/fogm/2015F/notes/intro.pdf), and as you can see it consists of a three step process which constantly repeats.

  1.  **Model:** Our model represents our beliefs about how the world works.  We want to consistantly return to the question "what process generated the data that I'm observing", and try to make sure that our model is as consistent with that our beliefs about that process as possible.
  2.  **Infer:** This is a fancy word for fitting a model with data.  
  3.  **Criticize:** Does out model make reasonable predictions?  Is it more accurate than a coin flip?
  
## Model Criticism

As we hinted above, one strategy for model criticism is to examine it's accuracy. What *accuracy* actually means depends on the outcome that we're trying to predict, but rest assured that there are good solutions for which ever outcome you choose. 

The results that we're getting  from python are actually summarizing a bunch of different accuracy measurements, so let's talk a little bit about what they mean. These are the results that are drawn from the top right section of the results summary.  

| Measure | Example Outcome | 
|----------------|-----------|
|R-squared| 0.01 | 
|Adj. R-squared| 0.009|
|F-statistic|3.990|
|Prob (F-statistic)|0.046|
|Log-Likelihood|-7164.6|
|AIC|1.433e+04|
|BIC|1.434e+04|

  * [$R^2$ and $\text{Adj. } R^2$](https://en.wikipedia.org/wiki/Coefficient_of_determination): More formally called the coefficient of determination, this values is the percentage of the outputs variance that's explained by the features of the model. Values can be between 0 and 1, with 1 being 100% of the variance of the output being explined by the features.  With $R^2 = 1$, our model should always make correct predictions. The *adjusted* value is very similar and tries to account for certain types of bias.  It will typically be very close to the $R^2$ value, but is a good one to look at when comparing models. 
  * [F-statistic and Prob (F-statistic)](https://en.wikipedia.org/wiki/F-test): This is a built in statistical test which essentially compares the model that you've proposed to one which has no features in it.  If the Prob (F-statistic) (i.e. the p-value) is sufficiently low, then you might believe that this model with its proposed features are better than a model without them. 
  * [Log-Likelihood](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation): Without going into detail, when the model is fit it is trying to maximize this value.  So between two models, the higher the log-likelihood the better the prediction accuracy. 
  * [AIC and BIC](https://en.wikipedia.org/wiki/Akaike_information_criterion): It's not so important how these are calculated as I don't think they offer any greater interpritability, but the smaller the value the better when choosing your model.



# Mini-Hackathon Time

With whatever time we have left your job is to explore the data and find the best sets of predictors for either Math or ELA assessment change. 

[And as usual, please leave me some feedback! It's appreciated :)](https://forms.office.com/Pages/ResponsePage.aspx?id=n4nHpSnR9kisiI-X82badMFC4tEHX8lCm8qe3Orb0kdUQkw2TkQxUFJEWjlCRElPVlJQWktVRlQ0QiQlQCN0PWcu) 